# Install Packages

In [ ]:
!pip install pandas
!pip install numpy
!pip install emoji
!pip install nltk
!pip install emoji
!pip install re
!pip install textblob
!pip install unicodedata
!pip install sklearn
!pip install imblearn


# Import dataset

In [ ]:
import numpy as np 
import pandas as pd 

#dataset with 10k comments
df = pd.read_csv('../input/just-dance-on-youtube/jd-dataset-topics-Human-Review.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('commentID', axis=1, inplace=True)
df.drop('expandedText', axis=1, inplace=True)
df.drop('Sexual activity', axis=1, inplace=True)

print(df.shape)

data = df
data.head()

## Define the topics to classify

In [ ]:
# define the 37 target topics of the dataset 
categories = ['Memorability', 'Learnability', 'Efficiency', 'Errors/Effectiveness', 'Satisfaction', 
			'Aesthetics and Appeal',
			'Affect and Emotion',
			'Anticipation',
			'Comfort',
			'Detailed Usability',
			'Enchantment',
			'Engagement',
			'Enjoyment and Fun',
			'Frustration',
			'Hedonic',
			'Impact',
			'Likeability', 
			'Motivation',
			'Overall Usability',
			'Pleasure',
			'Support',
			'Trust',
			'User Differences',
			'Bodily image and Appearance',
			'Concentration',
			'Energy',
			'Fatigue',
			'Learning',
			'Memory',
			'Negative feelings',
			'Pain and Discomfort', 
			'Personal relationships',
			'Positive feelings',
			'Self-esteem',
			#'Sexual activity',
			'Sleep and Rest',
			'Social support',
			'Thinking']

## **DATA PRE-PROCESSING** 




In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import emoji
import unicodedata
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
from textblob import TextBlob, Word

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw')

In [ ]:
def emojiToCLDRshortName(text):
	try:
		has_emoji = bool(emoji.get_emoji_regexp().search(text))
		if (has_emoji):
			emoji_chars = emoji.EMOJI_ALIAS_UNICODE_ENGLISH.values()
			def _emoji(char):
				if char in emoji_chars:
					return unicodedata.name(char) + " "
			return ''.join(_emoji(char) or char for char in text)
		else:
			return text
	except Exception as e:
		print("Emoji convert - ", e)


def contractions(text):
	# https://gist.github.com/nealrs/96342d8231b75cf4bb82
	# https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions

	cDict = {
	  "ain't": "am not",
	  "aren't": "are not",
	  "can't": "cannot",
	  "can't've": "cannot have",
	  "'cause": "because",
	  "could've": "could have",
	  "couldn't": "could not",
	  "couldn't've": "could not have",
	  "didn't": "did not",
	  "doesn't": "does not",
	  "don't": "do not",
	  "hadn't": "had not",
	  "hadn't've": "had not have",
	  "hasn't": "has not",
	  "haven't": "have not",
	  "he'd": "he would",
	  "he'd've": "he would have",
	  "he'll": "he will",
	  "he'll've": "he will have",
	  "he's": "he is",
	  "how'd": "how did",
	  "how'd'y": "how do you",
	  "how'll": "how will",
	  "how's": "how is",
	  "I'd": "I would",
	  "I'd've": "I would have",
	  "I'll": "I will",
	  "I'll've": "I will have",
	  "I'm": "I am",
	  "I've": "I have",
	  "isn't": "is not",
	  "it'd": "it had",
	  "it'd've": "it would have",
	  "it'll": "it will",
	  "it'll've": "it will have",
	  "it's": "it is",
	  "let's": "let us",
	  "ma'am": "madam",
	  "mayn't": "may not",
	  "might've": "might have",
	  "mightn't": "might not",
	  "mightn't've": "might not have",
	  "must've": "must have",
	  "mustn't": "must not",
	  "mustn't've": "must not have",
	  "needn't": "need not",
	  "needn't've": "need not have",
	  "o'clock": "of the clock",
	  "oughtn't": "ought not",
	  "oughtn't've": "ought not have",
	  "shan't": "shall not",
	  "sha'n't": "shall not",
	  "shan't've": "shall not have",
	  "she'd": "she would",
	  "she'd've": "she would have",
	  "she'll": "she will",
	  "she'll've": "she will have",
	  "she's": "she is",
	  "should've": "should have",
	  "shouldn't": "should not",
	  "shouldn't've": "should not have",
	  "so've": "so have",
	  "so's": "so is",
    "im":"i am",
	  "that'd": "that would",
	  "that'd've": "that would have",
	  "that's": "that is",
	  "thats": "that is",
	  "there'd": "there had",
	  "there'd've": "there would have",
	  "there's": "there is",
	  "they'd": "they would",
	  "they'd've": "they would have",
	  "they'll": "they will",
	  "they'll've": "they will have",
	  "they're": "they are",
	  "they've": "they have",
	  "to've": "to have",
	  "wasn't": "was not",
	  "we'd": "we had",
	  "we'd've": "we would have",
	  "we'll": "we will",
	  "we'll've": "we will have",
	  "we're": "we are",
	  "we've": "we have",
	  "weren't": "were not",
	  "what'll": "what will",
	  "what'll've": "what will have",
	  "what're": "what are",
	  "what's": "what is",
	  "what've": "what have",
	  "when's": "when is",
	  "when've": "when have",
	  "where'd": "where did",
	  "where's": "where is",
	  "where've": "where have",
	  "who'll": "who will",
	  "who'll've": "who will have",
	  "who's": "who is",
	  "who've": "who have",
	  "why's": "why is",
	  "why've": "why have",
	  "will've": "will have",
	  "won't": "will not",
	  "won't've": "will not have",
	  "would've": "would have",
	  "wouldn't": "would not",
	  "wouldn't've": "would not have",
	  "y'all": "you all",
	  "y'alls": "you alls",
	  "y'all'd": "you all would",
	  "y'all'd've": "you all would have",
	  "y'all're": "you all are",
	  "y'all've": "you all have",
	  "you'd": "you had",
	  "you'd've": "you would have",
	  "you'll": "you you will",
	  "you'll've": "you you will have",
	  "you're": "you are",
	  "you've": "you have"
	}
	try:
		c_re = re.compile('(%s)' % '|'.join(cDict.keys()))

		def expandContractions(text, c_re=c_re):
			def replace(match):
				return cDict[match.group(0)]
			return c_re.sub(replace, text)
		
		text = expandContractions(text.lower())
		return text
	except Exception as e:
		print("contractions", e)


def repeatedCharacters(text):
	# https://www.nltk.org/_modules/nltk/tokenize/casual.html#reduce_lengthening
	try:
		if (len(text) > 1):
			pattern = re.compile(r"(.)\1{2,}") #regex
			return pattern.sub(r"\1\1\1", text)
		else:
			return text
	except Exception as e:
		print("repeatedCharacters error - ", e)


def spellCorrection(text):
	try:
		text = TextBlob(text).correct()
	except Exception as e:
		print("spellCorrection error - ", e)
  
	return text

def slangs(text):
  try:
    file = 'https://raw.githubusercontent.com/renatojmsantos/An-Opinion-Mining-Approach-to-Analyse-Perceived-Impacts-of-Games-for-Health/main/acronimos.csv'
    acron = pd.read_csv(file,lineterminator='\n',encoding='utf-8')

    slang = acron['slang']
    complete = acron['complete']
    row = 0
    for s in slang:
      if(slang[row] in text.lower().strip().split()):
        text = text.lower()
        text = text.replace(slang[row],str(complete[row].strip()))
      else:
        row+=1
    return text
  except IOError as e:
    print(e)
  except FileNotFoundError:
    print("File not found...", file)
  except Exception as ex:
    print("slangs...", ex)
  
  
def clearText(text):
  try:
    # contractions
    #text = contractions(str(text))
    
    #acronyms
    #text = slangs(str(text))

    # emojis to cldr
    text = str(emojiToCLDRshortName(str(text)))
    # remove URLs
    text = re.sub('https?://[A-Za-z0-9./?&=_]+','',text)
    # hashtags
    text = re.sub('#[A-Za-z0-9]+','',text)
    # mentions
    text = re.sub('@[A-Za-z0-9._-]+','',text)
    # to lower
    text = text.lower()
    # remove pontuation
    text = re.sub(r"[^\w\s]","",text)
    #remove white spaces
    text = " ".join(text.strip().split())
    text = re.sub(r"[\W\s]"," ",text)
    text = re.sub("\n","",text)
    # replace just dance with game... important to consider dance as activity
    text = re.sub("just dance","game",text)
    #remove repeted characters
    text = repeatedCharacters(text)
    #do some spell correction
    #text = spellCorrection(text) 
  except Exception as e:
    print("clearText error - ", e)

  return text

def pos_tagger(nltk_tag):
  if nltk_tag.startswith('J'):
			return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  #elif nltk_tag.startswith('R'):
  #  return wordnet.ADV
  else:          
    return None

def stemmingText(text):
  #print(text)
  # get adj, verb, nouns
  textWords = word_tokenize(text)
  pos_tagged = nltk.pos_tag(textWords)
  
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

  #print(wordnet_tagged)
  # stemming
  stemmer = nltk.stem.SnowballStemmer('english') 
  stemSentence = ""
  for word, tag in wordnet_tagged:
    if (tag is not None):
      stem = stemmer.stem(word)
      stemSentence+=stem
      stemSentence += " "
  stemSentence = stemSentence.strip()
  #print(stemSentence)
  
  # remove stop words
  words = word_tokenize(stemSentence)
  stopwords = nltk.corpus.stopwords.words('english')
  pals = [word for word in words if not word in stopwords] 
  text = " ".join(pals)
  return text


In [ ]:

def runPreprocessing(text):
  text = clearText(text)
  text = stemmingText(str(text))
  return text

#example = "I love playing just dance with my friends, OMG It is VERYYY cool !!!! 😂 😂 😂😂😂"
#print(runPreprocessing(str(example)))

#data['originalText'] = data['originalText'].apply(runPreprocessing)
#data.head()



# TF-IDF VECTORIZER



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

train, test = train_test_split(data, random_state=20, test_size=0.20) #, shuffle=True
#print(train.shape, test.shape)
#print(train[categories].sum())
train_text = train['originalText']
test_text = test['originalText']

#vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer = TfidfVectorizer(preprocessor=runPreprocessing)

x_train = vectorizer.fit_transform(train_text)
x_test = vectorizer.transform(test_text)

#vectorizer.get_feature_names_out()

## Import Packages

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.svm import LinearSVC

import imblearn
from imblearn.over_sampling import SMOTE

# Model Classification & Handling with inbalance dataset & Predict new text

In [ ]:
Model = OneVsRestClassifier(LinearSVC())
#Model = OneVsRestClassifier(SVC(kernel = 'linear',C=0.01,gamma='auto'))
#Model = OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)

print("{:<30} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Topic','Accuracy','Recall','Precision','F-1 Score', 'ROC_AUC', 'Balanced'))

#new_text = "I love play just dance with friends"
new_text = "THIS GAME IS HORRIBLE!"
#new_text = "I am thinking about playing this with my friends and family. It is so FUN!"

#clean_text = runPreprocessing(new_text)
text_pred = vectorizer.transform([new_text])
topics_predicted = []

for category in categories:
    x = x_train
    y = train[category]
    
    #balance dataset with Synthetic minority oversampling
    smote = SMOTE()
    x_smote, y_smote = smote.fit_resample(x, y)
    
    Model.fit(x_smote, y_smote)

    y_pred = Model.predict(x_test)
    y_true = test[category]

    accuracy = "{:.3f}".format(accuracy_score(y_true, y_pred))
    recall = "{:.3f}".format(recall_score(y_true, y_pred, average='weighted'))
    precision = "{:.3f}".format(precision_score(y_true, y_pred, average='micro'))
    f1 = "{:.3f}".format(f1_score(y_true, y_pred, average='weighted'))
    roc_auc = "{:.3f}".format(roc_auc_score(y_true, y_pred))
    balanced = "{:.3f}".format(balanced_accuracy_score(y_true, y_pred))

    print("{:<30} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(str(category), accuracy, recall, precision, f1, roc_auc, balanced))
    #print(confusion_matrix(y_pred, y_true))
    
    #text_pred = vectorizer.transform([clean_text])
    if([1] in (Model.predict(text_pred).astype(int))):
      topics_predicted.append(category)

print("\nNEW TEXT: '",new_text,"'\nPREDICTED: ",topics_predicted)
